# Import packages:


In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

### Load time series data from different sets


In [ ]:
train_df = pd.read_csv("../data/raw/us_drought/train_timeseries.csv")
test_df = pd.read_csv("../data/raw/us_drought/test_timeseries.csv")
val_df = pd.read_csv("../data/raw/us_drought/validation_timeseries.csv")

### Concatenate datasets


In [ ]:
data = pd.concat([train_df, test_df, val_df])

In [ ]:
data.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2000-01-01,0.22,100.51,9.65,14.74,13.51,13.51,20.96,11.46,...,14.65,2.20,2.94,1.49,1.46,4.85,6.04,3.23,2.81,NaN
1,1001,2000-01-02,0.20,100.55,10.42,16.69,14.71,14.71,22.80,12.61,...,16.60,2.52,3.43,1.83,1.60,5.33,6.13,3.72,2.41,NaN
2,1001,2000-01-03,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,...,18.41,4.03,5.33,2.66,2.67,7.53,9.52,5.87,3.66,NaN
3,1001,2000-01-04,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,...,11.31,3.84,5.67,2.08,3.59,6.73,9.31,3.74,5.58,1.0
4,1001,2000-01-05,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,...,2.65,1.60,2.50,0.52,1.98,2.94,4.85,0.65,4.19,NaN


In [ ]:
data.dtypes

fips             int64
date            object
PRECTOT        float64
PS             float64
QV2M           float64
T2M            float64
T2MDEW         float64
T2MWET         float64
T2M_MAX        float64
T2M_MIN        float64
T2M_RANGE      float64
TS             float64
WS10M          float64
WS10M_MAX      float64
WS10M_MIN      float64
WS10M_RANGE    float64
WS50M          float64
WS50M_MAX      float64
WS50M_MIN      float64
WS50M_RANGE    float64
score          float64
dtype: object

### Process date column


Convert `date` colume to `datetime` type


In [ ]:
data["date"] = pd.to_datetime(data["date"])

### Standardize FIPS code


Pad `fips` code, making sure it's 5 digits:


In [ ]:
data["fips"] = data["fips"].astype("string[pyarrow]").str.zfill(5)

### Rename score column to drought


In [ ]:
data = data.rename(columns={"score": "drought"})

### Load US states shapefile data for geospatial mapping


In [ ]:
us_states = gpd.read_file("../data/raw/maps/cb_2022_us_state_500k.zip")

In [ ]:
us_states.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,35,00897535,0400000US35,35,NM,New Mexico,00,314198573403,726463825,"POLYGON ((-109.05017 31.48, -109.04984 31.4995..."
1,46,01785534,0400000US46,46,SD,South Dakota,00,196341552329,3387681983,"POLYGON ((-104.05788 44.9976, -104.05078 44.99..."
2,06,01779778,0400000US06,06,CA,California,00,403673617862,20291712025,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987..."
3,21,01779786,0400000US21,21,KY,Kentucky,00,102266581101,2384240769,"MULTIPOLYGON (((-89.40565 36.52816, -89.39868 ..."
4,01,01779775,0400000US01,01,AL,Alabama,00,131185042550,4582333181,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."


### Add state names to the data by merging with state shapefile data


In [ ]:
data = (
    data.assign(state_fip=data["fips"].str[:2])
    .drop(columns=["fips"])
    .merge(
        right=us_states[["STATEFP", "NAME"]],
        left_on="state_fip",
        right_on="STATEFP",
        how="inner",
    )
    .drop(columns=["state_fip", "STATEFP"])
    .rename(columns={"NAME": "state_name"})
)

In [ ]:
data.head()

,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,...,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,drought,state_name
0,2000-01-01,0.22,100.51,9.65,14.74,13.51,13.51,20.96,11.46,9.50,...,2.20,2.94,1.49,1.46,4.85,6.04,3.23,2.81,NaN,Alabama
1,2000-01-02,0.20,100.55,10.42,16.69,14.71,14.71,22.80,12.61,10.18,...,2.52,3.43,1.83,1.60,5.33,6.13,3.72,2.41,NaN,Alabama
2,2000-01-03,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,7.41,...,4.03,5.33,2.66,2.67,7.53,9.52,5.87,3.66,NaN,Alabama
3,2000-01-04,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,15.92,...,3.84,5.67,2.08,3.59,6.73,9.31,3.74,5.58,1.0,Alabama
4,2000-01-05,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,13.48,...,1.60,2.50,0.52,1.98,2.94,4.85,0.65,4.19,NaN,Alabama


### Spatio-temporal aggregation to create average monthly metrics by state


- Climatic factors are recorded daily.
- Drought values are recorded weekly.
- Each state has a various number of FIPS codes corresponding to its counties or county-equivalents.

Aggregating data from FIPS-level to state-level reduces granularity, enabling more efficient analysis with coarser, smaller datasets.

We aggregate the data to have monthly average values at the state-level:


In [ ]:
aggregated_data = data.groupby(by="state_name").resample(rule="ME", on="date").mean()

In [ ]:
aggregated_data.head()

PRECTOT          PS       QV2M        T2M     T2MDEW  \
state_name date                                                                
Alabama    2000-01-31  3.405874  100.431849   5.574343   7.086018   3.375691   
           2000-02-29  1.631266  100.491168   6.441529  10.378271   6.115095   
           2000-03-31  3.830448   99.953139   8.057650  14.412908   9.896991   
           2000-04-30  3.885766   99.886109   8.402418  15.811174  10.629338   
           2000-05-31  1.036216   99.836365  11.518902  23.823962  15.711555   

                          T2MWET    T2M_MAX    T2M_MIN  T2M_RANGE         TS  \
state_name date                                                                
Alabama    2000-01-31   3.422152  12.824882   1.824420  11.000318   6.947588   
           2000-02-29   6.132362  17.774056   3.886742  13.887401  10.126608   
           2000-03-31   9.904343  21.515753   7.732812  13.783105  14.240125   
           2000-04-30  10.636189  22.472393   9.099811  13.372667  15.707030   
           2000-05-31  15.713755  31.097838  17.021305  14.076384  23.940385   

                          WS10M  WS10M_MAX  WS10M_MIN  WS10M_RANGE     WS50M  \
state_name date                                                                
Alabama    2000-01-31  2.576543   3.605055   1.573062     2.032157  4.925084   
           2000-02-29  2.433294   3.451245   1.466315     1.984766  4.764323   
           2000-03-31  2.349605   3.274141   1.422008     1.851960  4.611343   
           2000-04-30  2.514234   3.627438   1.354468     2.272687  4.802886   
           2000-05-31  2.250852   3.200583   1.345407     1.855181  4.439254   

                       WS50M_MAX  WS50M_MIN  WS50M_RANGE   drought  
state_name date                                                     
Alabama    2000-01-31   6.681468   3.006625     3.674699  1.826632  
           2000-02-29   6.647730   2.706881     3.941081  1.576023  
           2000-03-31   6.370164   2.754766     3.615407  1.826211  
           2000-04-30   6.640905   2.653672     3.987249  1.268811  
           2000-05-31   5.982735   2.651411     3.331213  1.913529

### Save aggregated data to Parquet


In [ ]:
aggregated_data.to_parquet("../data/processed/aggregated_data.parquet")